 <a name="1"></a>
# <span style="color:green">Задачи исследования:<span>
    
- проанализировать тарифные планы компании
- проанализировать поведение клиентов компании использующие тарифные планы представленные ниже
- сделать вывод — какой тариф приносит больше денег.

### Описание тарифов

##### Тариф «Смарт»

1. Ежемесячная плата: 550 рублей
2. Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
3. Стоимость услуг сверх тарифного пакета:
    - минута разговора: 3 рубля («Мегалайн» всегда округляет вверх значения минут и мегабайтов.
    Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.)
    - сообщение: 3 рубля
    - 1 Гб интернет-трафика: 200 рублей

##### Тариф «Ультра»

1. Ежемесячная плата: 1950 рублей
2. Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
3. Стоимость услуг сверх тарифного пакета:
    - минута разговора: 1 рубль
    - сообщение: 1 рубль
    - 1 Гб интернет-трафика: 150 рублей

# План 

## 1. [Осмотр и изучение данных](#1)

## 2. [Предобработка данных](#2)

- исправление ошибок в данных
- проверка и предварительный анализ

## 3. [Подготовка, очистка и добавление данных](#3)

- индексы и названия столбцов
- работа с дубликатами
- работа с пропущенными значениями
- изменение типов данных
- добавление дополнительных метрик **для каждого пользователя:**
      
      
    - количество сделанных звонков и израсходованных минут разговора по месяцам;
    - количество отправленных сообщений по месяцам;
    - объем израсходованного интернет-трафика по месяцам;
    - помесячную выручку с каждого пользователя (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана).

## 4. [Исследовательский анализ данных](#4)

- Опишите поведение клиентов оператора, исходя из выборки.
- Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц?
- Посчитайте среднее количество, дисперсию и стандартное отклонение. Постройте гистограммы. Опишите распределения.(#41)
  
## 5. [Проверка гипотез](#5)

- средняя выручка пользователей тарифов «Ультра» и «Смарт» различается;
- средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов;
  
## 6. [Выводы и результаты исследования](#6)

# 1. Импорт библиотек и чтение данных

In [945]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Disable warnings in Anaconda
import warnings
warnings.filterwarnings('ignore')

from pylab import rcParams
rcParams['figure.figsize'] = 10,7
pd.set_option('display.max_columns', None)

In [946]:
calls = pd.read_csv('datasets/calls.csv')
internet = pd.read_csv('datasets/internet.csv')
messages = pd.read_csv('datasets/messages.csv')
tariffs = pd.read_csv('datasets/tariffs.csv')
users = pd.read_csv('datasets/users.csv')

### 1.1 Таблица звонков

In [947]:
calls.head(3)

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000


In [948]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
id           202607 non-null object
call_date    202607 non-null object
duration     202607 non-null float64
user_id      202607 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.2+ MB


### 1.2 Таблица интернет трафика

In [949]:
internet.head(3)

,id,mb_used,session_date,user_id
0,1000_0,112.95,2018-11-25,1000
1,1000_1,1052.81,2018-09-07,1000
2,1000_2,1197.26,2018-06-25,1000


In [950]:
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149396 entries, 0 to 149395
Data columns (total 4 columns):
id              149396 non-null object
mb_used         149396 non-null float64
session_date    149396 non-null object
user_id         149396 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.6+ MB


### 1.3 Таблица сообщений

In [951]:
messages.head(3)

,id,message_date,user_id
0,1000_0,2018-06-27,1000
1,1000_1,2018-10-08,1000
2,1000_2,2018-08-04,1000


In [952]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
id              123036 non-null object
message_date    123036 non-null object
user_id         123036 non-null int64
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


### 1.3 Таблица пользователей

In [953]:
users.head(3)

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart


In [954]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
user_id       500 non-null int64
age           500 non-null int64
churn_date    38 non-null object
city          500 non-null object
first_name    500 non-null object
last_name     500 non-null object
reg_date      500 non-null object
tariff        500 non-null object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


### 1.4 Таблица тарифов

In [955]:
tariffs.head()

,messages_included,mg_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra


### 1.5 Общая таблица данных

In [956]:
len(users), len(calls), len(messages), len(internet)

(500, 202607, 123036, 149396)

In [957]:
all_calls_user = calls.pivot_table(index='user_id', values='duration')
all_calls_user.head()

,duration
user_id,
1000,6.571304
1001,6.552951
1002,6.595253
1003,6.924077
1004,5.234148


In [958]:
all_internet_user = internet.pivot_table(index='user_id', values='mb_used')
all_internet_user.head()

,mb_used
user_id,
1000,508.968294
1001,338.187500
1002,314.462114
1003,550.775385
1004,496.798267


In [959]:
all_messages_user = messages.pivot_table(index='user_id', aggfunc='count')
all_messages_user.drop(['id'], axis=1, inplace=True)
all_messages_user.rename({'message_date': 'count_messages'}, axis=1, inplace=True)
all_messages_user.head()

,count_messages
user_id,
1000,496
1002,70
1003,380
1004,1192
1005,552


In [960]:
df = ( 
        users.merge(all_calls_user, how='outer', on='user_id')
             .merge(all_internet_user, how='outer', on='user_id')
             .merge(all_messages_user, how='outer', on='user_id')
      )

In [961]:
df.head()

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,duration,mb_used,count_messages
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,6.571304,508.968294,496.0
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart,6.552951,338.187500,NaN
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart,6.595253,314.462114,70.0
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra,6.924077,550.775385,380.0
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra,5.234148,496.798267,1192.0


In [962]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 11 columns):
user_id           500 non-null int64
age               500 non-null int64
churn_date        38 non-null object
city              500 non-null object
first_name        500 non-null object
last_name         500 non-null object
reg_date          500 non-null object
tariff            500 non-null object
duration          492 non-null float64
mb_used           497 non-null float64
count_messages    426 non-null float64
dtypes: float64(3), int64(2), object(6)
memory usage: 46.9+ KB


 <a name="2"></a>
# 2. Предобработка данных и предварительный анализ
[К оглавлению](#1)

### Распределение пользователей по тарифам

In [963]:
df['tariff'].value_counts()

smart    351
ultra    149
Name: tariff, dtype: int64

In [964]:
## дейсвтующих
df[df['churn_date'].isna()]['tariff'].value_counts()

smart    328
ultra    134
Name: tariff, dtype: int64

In [965]:
## прекративших пользоваться услугами
df[df['churn_date'].isna()]['tariff'].value_counts()

smart    328
ultra    134
Name: tariff, dtype: int64

### Звоники длительностью менее 1 минуты

In [966]:
calls[calls['duration'] < 1]

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
5,1000_5,2018-11-02,0.00,1000
6,1000_6,2018-10-18,0.00,1000
9,1000_9,2018-08-15,0.00,1000
...,...,...,...,...
202592,1499_205,2018-10-13,0.00,1499
202593,1499_206,2018-12-26,0.00,1499
202594,1499_207,2018-10-28,0.00,1499
202596,1499_209,2018-12-16,0.00,1499


### Вывод:
**Всего в нашей выборке 500 абонентов:**
- тариф smart 351 пользователя
- тариф ultra 149 пользователя

##### из которых действующих 462:

- тариф smart 328
- тариф ultra 134

##### прекративших пользоваться услугами 38**:
- тариф smart 23
- тариф ultra 15

**Предварительный анализ показал что:**

- Требуется изменить в данных тип данных всех дат в datetime
- Столбец **churn_date** можно изменить на True и False - по желанию
- Опереатор всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута. Значит требуется округлить значения меньше минуты 
- Интернет трафик по тарифу расчитывается в Гб а в таблице Мб. Обратить внимание при обобщении трафика для каждого usera 

<a name="3"></a>
# 3. Подготовка и очистка данных
[К оглавлению](#1)


### Заменяем пропущенные значения

In [967]:
users['churn_date'].fillna(False, inplace=True)

In [968]:
def split_charn(value):
    if value:
        return True
    return False

In [969]:
users['churn_date'] = users['churn_date'].apply(split_charn)
# users['churn_date'].value_counts()

### Изменяем тип данных в таблицах

In [970]:
def get_datetime(data, columns):
    for col in columns:
        data[col] = data[col].astype('datetime64[ns]')
    return data

In [971]:
get_datetime(calls, ['call_date'])
get_datetime(internet, ['session_date'])
get_datetime(messages, ['message_date'])
get_datetime(users, ['reg_date'])

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,False,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,False,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,False,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
3,1003,23,False,Москва,Белла,Белякова,2018-08-17,ultra
4,1004,68,False,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra
...,...,...,...,...,...,...,...,...
495,1495,65,False,Иркутск,Авксентий,Фокин,2018-08-28,ultra
496,1496,36,False,Вологда,Трифон,Блохин,2018-01-27,smart
497,1497,32,False,Челябинск,Каролина,Агеева,2018-10-09,smart
498,1498,68,True,Владикавказ,Всеволод,Акимчин,2018-07-19,smart


### Округлим данные по звонкам и заменим нули на 1

In [972]:
# Смотрим сколько данных меньше 1 минуты
calls[calls['duration'] < 1]

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
5,1000_5,2018-11-02,0.00,1000
6,1000_6,2018-10-18,0.00,1000
9,1000_9,2018-08-15,0.00,1000
...,...,...,...,...
202592,1499_205,2018-10-13,0.00,1499
202593,1499_206,2018-12-26,0.00,1499
202594,1499_207,2018-10-28,0.00,1499
202596,1499_209,2018-12-16,0.00,1499


In [973]:
# Приводим значения в порядок
calls['duration'] = calls['duration'].apply(np.ceil).astype('int').replace(0, 1)
calls.head()

,id,call_date,duration,user_id
0,1000_0,2018-07-25,1,1000
1,1000_1,2018-08-17,1,1000
2,1000_2,2018-06-11,3,1000
3,1000_3,2018-09-21,14,1000
4,1000_4,2018-12-15,6,1000


### Округлим данные интерент сессий, заменим нули на 1

In [974]:
# Смотрим данные по интернет сессиям
internet[internet['mb_used'] < 1]

,id,mb_used,session_date,user_id
11,1000_11,0.0,2018-08-28,1000
38,1000_38,0.0,2018-11-27,1000
46,1000_46,0.0,2018-06-30,1000
67,1000_67,0.0,2018-06-16,1000
72,1000_72,0.0,2018-06-15,1000
...,...,...,...,...
149380,1499_141,0.0,2018-12-11,1499
149381,1499_142,0.0,2018-11-01,1499
149383,1499_144,0.0,2018-11-27,1499
149386,1499_147,0.0,2018-10-21,1499


In [975]:
# Приводим значения в порядок
internet['mb_used'] = internet['mb_used'].apply(np.ceil).astype('int').replace(0, 1)
internet

,id,mb_used,session_date,user_id
0,1000_0,113,2018-11-25,1000
1,1000_1,1053,2018-09-07,1000
2,1000_2,1198,2018-06-25,1000
3,1000_3,551,2018-08-22,1000
4,1000_4,303,2018-09-24,1000
...,...,...,...,...
149391,1499_152,319,2018-10-03,1499
149392,1499_153,491,2018-12-14,1499
149393,1499_154,1,2018-10-27,1499
149394,1499_155,1247,2018-11-26,1499


## 3.2 Добавление дополнительных метрик для пользователей:
- количество сделанных звонков и израсходованных минут разговора по месяцам;
- количество отправленных сообщений по месяцам;
- объем израсходованного интернет-трафика по месяцам;
- помесячную выручку с каждого пользователя (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана).

### 3.2.1 Количество сделанных звонков и израсходованных минут разговора по месяцам

In [976]:
calls['call_date_month'] = calls['call_date'].dt.month_name()

In [977]:
calls.head(1)

,id,call_date,duration,user_id,call_date_month
0,1000_0,2018-07-25,1,1000,July


### Варианты группировок для извлечения данных 
- Существует множество вариантов для группировки и создания таблиц, но для дальнейшего обучения моделей лучше выделять признаки в отдельный столбец

In [978]:
# calls.pivot_table(
#     index=['call_date_month', 'user_id'],
#     values='duration',
#     aggfunc=['count', 'sum'])


In [979]:
# calls.groupby(['user_id', 'call_date_month'])\
#                   .agg({'duration':'sum', 'id':'count'})\
#                   .reset_index()\
#                   .rename(columns={'duration':'month_calls_duration','id':'calls_total_in_month'})\

In [980]:
## Временный датасет с количеством минут и разбиением по месяцам
temp = calls.pivot_table(index='user_id', 
                    columns='call_date_month', 
                    values='duration',
                    aggfunc=['count', 'sum'],
                    fill_value = 0)
temp 

count                                                       \
call_date_month April August December February January July June March May   
user_id                                                                      
1000                0     52       46        0       0   47   43     0  22   
1001                0      0       63        0       0    0    0     0   0   
1002                0     42       33        0       0   26   15     0   0   
1003                0     55      108        0       0    0    0     0   0   
1004                0     19       21        0       0   22   31     0   9   
...               ...    ...      ...      ...     ...  ...  ...   ...  ..   
1495                0     17      155        0       0    0    0     0   0   
1496               67     88       80       77       7   73   74    79  63   
1497                0      0       71        0       0    0    0     0   0   
1498                0     57        0        0       0   23    0     0   0   
1499                0      0       69        0       0    0    0     0   0   

                                             sum                           \
call_date_month November October September April August December February   
user_id                                                                     
1000                  43      57        58     0    418      341        0   
1001                  59       0         0     0      0      430        0   
1002                  32      33        36     0    294      242        0   
1003                 115     108       134     0    392      822        0   
1004                  19      29        26     0    138      136        0   
...                  ...     ...       ...   ...    ...      ...      ...   
1495                 129     152       112     0     96     1066        0   
1496                  80      72        68   536    553      533      606   
1497                  64      66         0     0      0      558        0   
1498                   0      41        59     0    414        0        0   
1499                  74      68         9     0      0      505        0   

                                                                         
call_date_month January July June March  May November October September  
user_id                                                                  
1000                  0  346  187     0  164      344     363       478  
1001                  0    0    0     0    0      444       0         0  
1002                  0  219  118     0    0      250     221       217  
1003                  0    0    0     0    0      844     866       990  
1004                  0  137  177     0   37      121     153       125  
...                 ...  ...  ...   ...  ...      ...     ...       ...  
1495                  0    0    0     0    0      971    1056       830  
1496                 49  592  572   569  497      676     466       496  
1497                  0    0    0     0    0      387     506         0  
1498                  0  178    0     0    0        0     258       477  
1499                  0    0    0     0    0      624     473        71  

[492 rows x 24 columns]

In [981]:
#### Временный датасет с количеством минут и разбиением по месяцам + добавлены общие значения
temp_two = (         
            calls.pivot_table(index='user_id', 
                              values='duration',
                              aggfunc=['count', 'sum'],
                              fill_value = 0)
            .merge(temp, how='outer', on='user_id')
            )
temp_two

count      sum count                                             \
        duration duration April August December February January July June   
user_id                                                                      
1000         368     2641     0     52       46        0       0   47   43   
1001         122      874     0      0       63        0       0    0    0   
1002         217     1561     0     42       33        0       0   26   15   
1003         520     3914     0     55      108        0       0    0    0   
1004         176     1024     0     19       21        0       0   22   31   
...          ...      ...   ...    ...      ...      ...     ...  ...  ...   
1495         565     4019     0     17      155        0       0    0    0   
1496         828     6145    67     88       80       77       7   73   74   
1497         201     1451     0      0       71        0       0    0    0   
1498         180     1327     0     57        0        0       0   23    0   
1499         220     1673     0      0       69        0       0    0    0   

                                               sum                           \
        March May November October September April August December February   
user_id                                                                       
1000        0  22       43      57        58     0    418      341        0   
1001        0   0       59       0         0     0      0      430        0   
1002        0   0       32      33        36     0    294      242        0   
1003        0   0      115     108       134     0    392      822        0   
1004        0   9       19      29        26     0    138      136        0   
...       ...  ..      ...     ...       ...   ...    ...      ...      ...   
1495        0   0      129     152       112     0     96     1066        0   
1496       79  63       80      72        68   536    553      533      606   
1497        0   0       64      66         0     0      0      558        0   
1498        0   0        0      41        59     0    414        0        0   
1499        0   0       74      68         9     0      0      505        0   

                                                                 
        January July June March  May November October September  
user_id                                                          
1000          0  346  187     0  164      344     363       478  
1001          0    0    0     0    0      444       0         0  
1002          0  219  118     0    0      250     221       217  
1003          0    0    0     0    0      844     866       990  
1004          0  137  177     0   37      121     153       125  
...         ...  ...  ...   ...  ...      ...     ...       ...  
1495          0    0    0     0    0      971    1056       830  
1496         49  592  572   569  497      676     466       496  
1497          0    0    0     0    0      387     506         0  
1498          0  178    0     0    0        0     258       477  
1499          0    0    0     0    0      624     473        71  

[492 rows x 26 columns]

### Привяжем данные к пользователям 

- далее в данную таблицу смержим данные по смс и интернет трафику

In [982]:
full_users = users.merge(temp_two, how='outer', on='user_id')
full_users

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,"(count, duration)","(sum, duration)","(count, April)","(count, August)","(count, December)","(count, February)","(count, January)","(count, July)","(count, June)","(count, March)","(count, May)","(count, November)","(count, October)","(count, September)","(sum, April)","(sum, August)","(sum, December)","(sum, February)","(sum, January)","(sum, July)","(sum, June)","(sum, March)","(sum, May)","(sum, November)","(sum, October)","(sum, September)"
0,1000,52,False,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,368.0,2641.0,0.0,52.0,46.0,0.0,0.0,47.0,43.0,0.0,22.0,43.0,57.0,58.0,0.0,418.0,341.0,0.0,0.0,346.0,187.0,0.0,164.0,344.0,363.0,478.0
1,1001,41,False,Москва,Иван,Ежов,2018-11-01,smart,122.0,874.0,0.0,0.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,430.0,0.0,0.0,0.0,0.0,0.0,0.0,444.0,0.0,0.0
2,1002,59,False,Стерлитамак,Евгений,Абрамович,2018-06-17,smart,217.0,1561.0,0.0,42.0,33.0,0.0,0.0,26.0,15.0,0.0,0.0,32.0,33.0,36.0,0.0,294.0,242.0,0.0,0.0,219.0,118.0,0.0,0.0,250.0,221.0,217.0
3,1003,23,False,Москва,Белла,Белякова,2018-08-17,ultra,520.0,3914.0,0.0,55.0,108.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0,108.0,134.0,0.0,392.0,822.0,0.0,0.0,0.0,0.0,0.0,0.0,844.0,866.0,990.0
4,1004,68,False,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra,176.0,1024.0,0.0,19.0,21.0,0.0,0.0,22.0,31.0,0.0,9.0,19.0,29.0,26.0,0.0,138.0,136.0,0.0,0.0,137.0,177.0,0.0,37.0,121.0,153.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,65,False,Иркутск,Авксентий,Фокин,2018-08-28,ultra,565.0,4019.0,0.0,17.0,155.0,0.0,0.0,0.0,0.0,0.0,0.0,129.0,152.0,112.0,0.0,96.0,1066.0,0.0,0.0,0.0,0.0,0.0,0.0,971.0,1056.0,830.0
496,1496,36,False,Вологда,Трифон,Блохин,2018-01-27,smart,828.0,6145.0,67.0,88.0,80.0,77.0,7.0,73.0,74.0,79.0,63.0,80.0,72.0,68.0,536.0,553.0,533.0,606.0,49.0,592.0,572.0,569.0,497.0,676.0,466.0,496.0
497,1497,32,False,Челябинск,Каролина,Агеева,2018-10-09,smart,201.0,1451.0,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,66.0,0.0,0.0,0.0,558.0,0.0,0.0,0.0,0.0,0.0,0.0,387.0,506.0,0.0
498,1498,68,True,Владикавказ,Всеволод,Акимчин,2018-07-19,smart,180.0,1327.0,0.0,57.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,41.0,59.0,0.0,414.0,0.0,0.0,0.0,178.0,0.0,0.0,0.0,0.0,258.0,477.0


### 3.2.2 Количество отправленных сообщений по месяцам

In [983]:
messages['message_date_month'] = messages['message_date'].dt.month_name()
messages.head(1)
# messages['count'] = 1

,id,message_date,user_id,message_date_month
0,1000_0,2018-06-27,1000,June


In [984]:
# Группируем по месяцам количество отправленных сообщений
temp = messages.pivot_table(index='user_id', 
                            columns='message_date_month',
                            values='message_date',
                            aggfunc='count',
                            fill_value = 0
                           )
temp

message_date_month,April,August,December,February,January,July,June,March,May,November,October,September
user_id,,,,,,,,,,,,
1000,0,81,70,0,0,75,60,0,22,58,73,57
1002,0,13,12,0,0,11,4,0,0,16,10,4
1003,0,37,75,0,0,0,0,0,0,94,83,91
1004,0,151,162,0,0,181,134,0,95,158,165,146
1005,41,64,54,38,7,56,47,56,42,51,44,52
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,13,84,0,0,0,0,0,0,79,82,82
1496,49,41,51,45,9,42,35,46,40,46,35,29
1497,0,0,17,0,0,0,0,0,0,14,20,0


In [985]:
# Группируем общее количество смс пользователя
temp_two = messages.pivot_table(index='user_id', values='message_date', aggfunc='count')\
           .rename(columns={'message_date':'count_message'})
temp_two

,count_message
user_id,
1000,496
1002,70
1003,380
1004,1192
1005,552
...,...
1495,340
1496,468
1497,51


In [986]:
# мержим временные данные по сообщениям
temp_three = temp_two.merge(temp, how='outer', on='user_id')
temp_three

,count_message,April,August,December,February,January,July,June,March,May,November,October,September
user_id,,,,,,,,,,,,,
1000,496,0,81,70,0,0,75,60,0,22,58,73,57
1002,70,0,13,12,0,0,11,4,0,0,16,10,4
1003,380,0,37,75,0,0,0,0,0,0,94,83,91
1004,1192,0,151,162,0,0,181,134,0,95,158,165,146
1005,552,41,64,54,38,7,56,47,56,42,51,44,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,340,0,13,84,0,0,0,0,0,0,79,82,82
1496,468,49,41,51,45,9,42,35,46,40,46,35,29
1497,51,0,0,17,0,0,0,0,0,0,14,20,0


### Мержим данные в гланую таблицу

In [987]:
full_users = full_users.merge(temp_three, how='outer', on='user_id')
full_users

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,"(count, duration)","(sum, duration)","(count, April)","(count, August)","(count, December)","(count, February)","(count, January)","(count, July)","(count, June)","(count, March)","(count, May)","(count, November)","(count, October)","(count, September)","(sum, April)","(sum, August)","(sum, December)","(sum, February)","(sum, January)","(sum, July)","(sum, June)","(sum, March)","(sum, May)","(sum, November)","(sum, October)","(sum, September)",count_message,April,August,December,February,January,July,June,March,May,November,October,September
0,1000,52,False,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,368.0,2641.0,0.0,52.0,46.0,0.0,0.0,47.0,43.0,0.0,22.0,43.0,57.0,58.0,0.0,418.0,341.0,0.0,0.0,346.0,187.0,0.0,164.0,344.0,363.0,478.0,496.0,0.0,81.0,70.0,0.0,0.0,75.0,60.0,0.0,22.0,58.0,73.0,57.0
1,1001,41,False,Москва,Иван,Ежов,2018-11-01,smart,122.0,874.0,0.0,0.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,430.0,0.0,0.0,0.0,0.0,0.0,0.0,444.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1002,59,False,Стерлитамак,Евгений,Абрамович,2018-06-17,smart,217.0,1561.0,0.0,42.0,33.0,0.0,0.0,26.0,15.0,0.0,0.0,32.0,33.0,36.0,0.0,294.0,242.0,0.0,0.0,219.0,118.0,0.0,0.0,250.0,221.0,217.0,70.0,0.0,13.0,12.0,0.0,0.0,11.0,4.0,0.0,0.0,16.0,10.0,4.0
3,1003,23,False,Москва,Белла,Белякова,2018-08-17,ultra,520.0,3914.0,0.0,55.0,108.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0,108.0,134.0,0.0,392.0,822.0,0.0,0.0,0.0,0.0,0.0,0.0,844.0,866.0,990.0,380.0,0.0,37.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,83.0,91.0
4,1004,68,False,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra,176.0,1024.0,0.0,19.0,21.0,0.0,0.0,22.0,31.0,0.0,9.0,19.0,29.0,26.0,0.0,138.0,136.0,0.0,0.0,137.0,177.0,0.0,37.0,121.0,153.0,125.0,1192.0,0.0,151.0,162.0,0.0,0.0,181.0,134.0,0.0,95.0,158.0,165.0,146.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,65,False,Иркутск,Авксентий,Фокин,2018-08-28,ultra,565.0,4019.0,0.0,17.0,155.0,0.0,0.0,0.0,0.0,0.0,0.0,129.0,152.0,112.0,0.0,96.0,1066.0,0.0,0.0,0.0,0.0,0.0,0.0,971.0,1056.0,830.0,340.0,0.0,13.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,82.0,82.0
496,1496,36,False,Вологда,Трифон,Блохин,2018-01-27,smart,828.0,6145.0,67.0,88.0,80.0,77.0,7.0,73.0,74.0,79.0,63.0,80.0,72.0,68.0,536.0,553.0,533.0,606.0,49.0,592.0,572.0,569.0,497.0,676.0,466.0,496.0,468.0,49.0,41.0,51.0,45.0,9.0,42.0,35.0,46.0,40.0,46.0,35.0,29.0
497,1497,32,False,Челябинск,Каролина,Агеева,2018-10-09,smart,201.0,1451.0,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,66.0,0.0,0.0,0.0,558.0,0.0,0.0,0.0,0.0,0.0,0.0,387.0,506.0,0.0,51.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,20.0,0.0
498,1498,68,True,Владикавказ,Всеволод,Акимчин,2018-07-19,smart,180.0,1327.0,0.0,57.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,41.0,59.0,0.0,414.0,0.0,0.0,0.0,178.0,0.0,0.0,0.0,0.0,258.0,477.0,168.0,0.0,62.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,42.0,44.0


### 3.2.3 Объем израсходованного интернет-трафика по месяцам

In [988]:
internet.head()

,id,mb_used,session_date,user_id
0,1000_0,113,2018-11-25,1000
1,1000_1,1053,2018-09-07,1000
2,1000_2,1198,2018-06-25,1000
3,1000_3,551,2018-08-22,1000
4,1000_4,303,2018-09-24,1000


In [989]:
internet['session_date_month'] = internet['session_date'].dt.month_name()
internet.head(1)

,id,mb_used,session_date,user_id,session_date_month
0,1000_0,113,2018-11-25,1000,November


In [990]:
## Группируем трафик пользователя по месяцам
temp = internet.pivot_table(index='user_id', 
                            columns='session_date_month',
                            values='mb_used',
                            aggfunc='sum',
                            fill_value = 0
                           )
temp

session_date_month,April,August,December,February,January,July,June,March,May,November,October,September
user_id,,,,,,,,,,,,
1000,0,14072,9829,0,0,14017,23262,0,2257,14769,14717,14583
1001,0,0,14059,0,0,0,0,0,0,18460,0,0
1002,0,20357,18147,0,0,17615,10878,0,0,18621,13913,16724
1003,0,8572,10132,0,0,0,0,0,0,11369,14779,12479
1004,0,28609,18041,0,0,22246,17621,0,13415,15634,18498,15130
...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,2616,23451,0,0,0,0,0,0,38973,29399,33107
1496,21113,23569,20770,28537,3768,24406,20175,24999,23751,25113,28227,26982
1497,0,0,19946,0,0,0,0,0,0,14180,11516,0


In [991]:
# Группируем общее количество трафика пользователя
temp_two = internet.pivot_table(index='user_id', values='mb_used', aggfunc='sum')\
           .rename(columns={'mb_used':'all_used_traffic'})
temp_two.head(1)

,all_used_traffic
user_id,
1000,107506


In [992]:
# Мержим временные таблицы трафика

In [993]:
temp_four = temp.merge(temp_two, how='outer', on='user_id')
temp_four

,April,August,December,February,January,July,June,March,May,November,October,September,all_used_traffic
user_id,,,,,,,,,,,,,
1000,0,14072,9829,0,0,14017,23262,0,2257,14769,14717,14583,107506
1001,0,0,14059,0,0,0,0,0,0,18460,0,0,32519
1002,0,20357,18147,0,0,17615,10878,0,0,18621,13913,16724,116255
1003,0,8572,10132,0,0,0,0,0,0,11369,14779,12479,57331
1004,0,28609,18041,0,0,22246,17621,0,13415,15634,18498,15130,149194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0,2616,23451,0,0,0,0,0,0,38973,29399,33107,127546
1496,21113,23569,20770,28537,3768,24406,20175,24999,23751,25113,28227,26982,271410
1497,0,0,19946,0,0,0,0,0,0,14180,11516,0,45642


In [994]:
full_users = full_users.merge(temp_four, how='outer', on='user_id', suffixes=('_message', '_internet'))
full_users

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,"(count, duration)","(sum, duration)","(count, April)","(count, August)","(count, December)","(count, February)","(count, January)","(count, July)","(count, June)","(count, March)","(count, May)","(count, November)","(count, October)","(count, September)","(sum, April)","(sum, August)","(sum, December)","(sum, February)","(sum, January)","(sum, July)","(sum, June)","(sum, March)","(sum, May)","(sum, November)","(sum, October)","(sum, September)",count_message,April_message,August_message,December_message,February_message,January_message,July_message,June_message,March_message,May_message,November_message,October_message,September_message,April_internet,August_internet,December_internet,February_internet,January_internet,July_internet,June_internet,March_internet,May_internet,November_internet,October_internet,September_internet,all_used_traffic
0,1000,52,False,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,368.0,2641.0,0.0,52.0,46.0,0.0,0.0,47.0,43.0,0.0,22.0,43.0,57.0,58.0,0.0,418.0,341.0,0.0,0.0,346.0,187.0,0.0,164.0,344.0,363.0,478.0,496.0,0.0,81.0,70.0,0.0,0.0,75.0,60.0,0.0,22.0,58.0,73.0,57.0,0.0,14072.0,9829.0,0.0,0.0,14017.0,23262.0,0.0,2257.0,14769.0,14717.0,14583.0,107506.0
1,1001,41,False,Москва,Иван,Ежов,2018-11-01,smart,122.0,874.0,0.0,0.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,430.0,0.0,0.0,0.0,0.0,0.0,0.0,444.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,14059.0,0.0,0.0,0.0,0.0,0.0,0.0,18460.0,0.0,0.0,32519.0
2,1002,59,False,Стерлитамак,Евгений,Абрамович,2018-06-17,smart,217.0,1561.0,0.0,42.0,33.0,0.0,0.0,26.0,15.0,0.0,0.0,32.0,33.0,36.0,0.0,294.0,242.0,0.0,0.0,219.0,118.0,0.0,0.0,250.0,221.0,217.0,70.0,0.0,13.0,12.0,0.0,0.0,11.0,4.0,0.0,0.0,16.0,10.0,4.0,0.0,20357.0,18147.0,0.0,0.0,17615.0,10878.0,0.0,0.0,18621.0,13913.0,16724.0,116255.0
3,1003,23,False,Москва,Белла,Белякова,2018-08-17,ultra,520.0,3914.0,0.0,55.0,108.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0,108.0,134.0,0.0,392.0,822.0,0.0,0.0,0.0,0.0,0.0,0.0,844.0,866.0,990.0,380.0,0.0,37.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,83.0,91.0,0.0,8572.0,10132.0,0.0,0.0,0.0,0.0,0.0,0.0,11369.0,14779.0,12479.0,57331.0
4,1004,68,False,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra,176.0,1024.0,0.0,19.0,21.0,0.0,0.0,22.0,31.0,0.0,9.0,19.0,29.0,26.0,0.0,138.0,136.0,0.0,0.0,137.0,177.0,0.0,37.0,121.0,153.0,125.0,1192.0,0.0,151.0,162.0,0.0,0.0,181.0,134.0,0.0,95.0,158.0,165.0,146.0,0.0,28609.0,18041.0,0.0,0.0,22246.0,17621.0,0.0,13415.0,15634.0,18498.0,15130.0,149194.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,65,False,Иркутск,Авксентий,Фокин,2018-08-28,ultra,565.0,4019.0,0.0,17.0,155.0,0.0,0.0,0.0,0.0,0.0,0.0,129.0,152.0,112.0,0.0,96.0,1066.0,0.0,0.0,0.0,0.0,0.0,0.0,971.0,1056.0,830.0,340.0,0.0,13.0,84.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,82.0,82.0,0.0,2616.0,23451.0,0.0,0.0,0.0,0.0,0.0,0.0,38973.0,29399.0,33107.0,127546.0
496,1496,36,False,Вологда,Трифон,Блохин,2018-01-27,smart,828.0,6145.0,67.0,88.0,80.0,77.0,7.0,73.0,74.0,79.0,63.0,80.0,72.0,68.0,536.0,553.0,533.0,606.0,49.0,592.0,572.0,569.0,497.0,676.0,466.0,496.0,468.0,49.0,41.0,51.0,45.0,9.0,42.0,35.0,46.0,40.0,46.0,35.0,29.0,21113.0,23569.0,20770.0,28537.0,3768.0,24406.0,20175.0,24999.0,23751.0,25113.0,28227.0,26982.0,271410.0
497,1497,32,False,Челябинск,Каролина,Агеева,2018-10-09,smart,201.0,1451.0,0.0,0.0,71.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,66.0,0.0,0.0,0.0,558.0,0.0,0.0,0.0,0.0,0.0,0.0,387.0,506.0,0.0,51.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,20.0,0.0,0.0,0.0,19946.0,0.0,0.0,0.0,0.0,0.0,0.0,14180.0,11516.0,0.0,45642.0
498,1498,68,True,Владикавказ,Всеволод,Акимчин,2018-07-19,smart,180.0,1327.0,0.0,57.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,41.0,59.0,0.0,414.0,0.0,0.0,0.0,178.0,0.0,0.0,0.0,0.0,258.0,477.0,168.0,0.0,62.0,0.0,0.0,0.0,

### 3.2.4 Помесячная выручка с каждого пользователя

- (вычтите бесплатный лимит из суммарного количества звонков, сообщений и интернет-трафика; остаток умножьте на значение из тарифного плана).

In [1005]:
full_users.columns

Index([             'user_id',                  'age',           'churn_date',
                       'city',           'first_name',            'last_name',
                   'reg_date',               'tariff',  ('count', 'duration'),
          ('sum', 'duration'),     ('count', 'April'),    ('count', 'August'),
        ('count', 'December'),  ('count', 'February'),   ('count', 'January'),
            ('count', 'July'),      ('count', 'June'),     ('count', 'March'),
             ('count', 'May'),  ('count', 'November'),   ('count', 'October'),
       ('count', 'September'),       ('sum', 'April'),      ('sum', 'August'),
          ('sum', 'December'),    ('sum', 'February'),     ('sum', 'January'),
              ('sum', 'July'),        ('sum', 'June'),       ('sum', 'March'),
               ('sum', 'May'),    ('sum', 'November'),     ('sum', 'October'),
         ('sum', 'September'),        'count_message',        'April_message',
             'August_message',     'December_message

In [1013]:
full_users[['user_id', ('sum', 'duration'), 'count_message', 'all_used_traffic' ]]

,user_id,"(sum, duration)",count_message,all_used_traffic
0,1000,2641.0,496.0,107506.0
1,1001,874.0,NaN,32519.0
2,1002,1561.0,70.0,116255.0
3,1003,3914.0,380.0,57331.0
4,1004,1024.0,1192.0,149194.0
...,...,...,...,...
495,1495,4019.0,340.0,127546.0
496,1496,6145.0,468.0,271410.0
497,1497,1451.0,51.0,45642.0
498,1498,1327.0,168.0,74567.0
